# Python Scripts to Control the ASG Module

As it is well known, Python can be used to access the RedPitaya registers by using the *periphery module*. We control the ASG module on the RedPitaya using a simple Python script.  

## HW Configuration 

Connect IN1 to OUT1 on your RedPitaya board and the range jumper on IN1 to LV. If the *periphery* module is nor installed you may install it with the *!pip install periphery* command line on a new cell.

We will make extensive use the RedPitaya memory map, it  may be consulted here:

In [1]:
from IPython.display import IFrame
IFrame("https://redpitaya.readthedocs.io/en/latest/developerGuide/software/build/fpga/regset_common.html", width=900, height=300)

### Arbitrary Signal in Burst Mode with the following parameters:

- frequency : 1 MHz

- One cycles per burst

- Cycle duration of 1 us (f = 1 MHz)

- One replica delayed 2 us

- Repeated until stopped at a rate of 1 Hz



In [ ]:
#!/usr/bin/python
######################################## inputs
CHANNEL = 1 # 1 
f = 1000000 # Hz
n_cycles = 1 # each cycle of 1 us
n_replicas = 1
delay = 2 # us after the end of the 2 cycles
period = 1 # second
########################################
from periphery import MMIO
import time
import numpy as np
IP_BASE_ADDRESS = 0x40200000 # ASG

######################################## # define wave

def wave():
    Y=[]
    w = 2**14
    tau = w/10
    x1 = w/2
    for x in range(w):
        if x<x1:y = np.exp(-x/tau) 
        else : y = .5*np.exp(-(x-x1)/tau) 
        Y.append(-y)  
    return Y
######################################## Stop DAC1
ADDRESS_RANGE = 0x24
regset = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
regset.write16(0,0x80) # Stop DAC1
######################################## calculate step
BUFFER_SIZE=16384
step = int(2**16 * f /(125000000 / BUFFER_SIZE))
print ("Time per cycle =", 1/f, " us")
######################################## write step for appropiate frequency
regset.write32(0x10,step) # write new step
######################################## configure burst structure 
regset.write16(0x18,n_cycles) # number of read cycles in one burst
regset.write16(0x1C,n_replicas) # number of burst repetitions 
regset.write32(0x20,delay) # delay between burst repetitions in us
######################################## Copy wave to memory register
IP_BASE_ADDRESS += 0x10000 # Ch A memory data (16k samples)
ADDRESS_RANGE = 0x1FFFC - 0x10000 # 65532 = 16k * 4
regset = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
V_ASG = wave()
for adr in range(0,0x1FFFC-0x10000,0x4): # 65532
    i=int(adr/4)
    DATA_VOLT = V_ASG[i]
    if (DATA_VOLT > 1): DATA_VOLT = 1
    if (DATA_VOLT < -1): DATA_VOLT = -1
    if (DATA_VOLT>=0):
        DATA = int(np.round((DATA_VOLT)*2**13))-1
    else:
        DATA = int(np.round((1+DATA_VOLT)*2**13)+2**13) 
    if (DATA > 2**14-1): DATA = 2**14-1
    if (DATA < 0): DATA = 0
    regset.write16(adr,DATA)
regset = MMIO(0x40200000, 0x2)
n = 0
while(1):
    n += 1
    regset.write16(0,1) # Start DAC1 in 1-trig immediately configuration
    time.sleep(period)
    regset.write16(0,0)
    print(n,end=" ")
regset.close()


Time per cycle = 1e-06  us
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 2